In [ ]:
from google.colab import files
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
# initial read of input csv files with converters

rawExcess = pd.read_csv('/content/Copy of MG-M0076-Excess_Inventory_-_18_Week.csv', thousands = ',')
rawDemand = pd.read_csv('/content/MG-O0717-DRP_Branch_Reference.csv', thousands = ',')

# data cleaning processes

# trim column names

rawExcess.columns = rawExcess.columns.str.strip()
rawDemand.columns = rawDemand.columns.str.strip()

# remove dollar signs

rawExcess['Standard Cost'] = rawExcess['Standard Cost'].str.replace(',', '').str.replace('$', '')

# NaN to 0 for later inequalities

rawExcess = rawExcess.fillna(0)
rawDemand = rawDemand.fillna(0)


# convert data types


# for excess
rawExcess['Supply - Open P.O. Qty'] = pd.to_numeric(rawExcess['Supply - Open P.O. Qty'],errors = 'raise')
rawExcess['Supply_18 Wk'] = pd.to_numeric(rawExcess['Supply_18 Wk'],errors = 'raise')
rawExcess['Demand_18 Wk'] = pd.to_numeric(rawExcess['Demand_18 Wk'],errors = 'raise')
rawExcess['Excess Qty_18 Wk'] = pd.to_numeric(rawExcess['Excess Qty_18 Wk'],errors = 'raise')
rawExcess['Standard Cost'] = pd.to_numeric(rawExcess['Standard Cost'],errors = 'raise')

# for demand

rawDemand['Request Date'] = pd.to_datetime(rawDemand['Request Date'],errors='ignore')
rawDemand['Trans QTY'] = np.floor(rawDemand['Trans QTY']).astype(int)
rawDemand['Header Branch'] = np.floor(rawDemand['Header Branch']).astype(int)

<ipython-input-10-b7206002fddc>:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rawExcess['Standard Cost'] = rawExcess['Standard Cost'].str.replace(',', '').str.replace('$', '')


In [ ]:
# roll up values in Excess to reduce unneccesary columns/observations

# define what you want to count as 'True Excess' 

columns = ['Plan Fmly','Supply - Open P.O. Qty','Demand_18 Wk','Excess Qty_18 Wk']
conditions = [(rawExcess['Plan Fmly'] != 'M23') & (rawExcess['Supply - Open P.O. Qty']<= 0) & (rawExcess['Demand_18 Wk'] <= 0) & (rawExcess['Excess Qty_18 Wk'] > 0)]

# remove CSM, DCL and PNT

rawExcess = rawExcess[~rawExcess['Part'].str.contains('CSM')]
rawExcess = rawExcess[~rawExcess['Part Description'].str.contains('DCL')]
rawExcess = rawExcess[~rawExcess['Part Description'].str.contains('PNT')]

# use boolean indexing to filter the Dataframe to only include 'True Excess'

rawExcess = rawExcess[conditions[0]]

# finally remove unneccesary columns and add columns to hold later calculations

excess = rawExcess[['Branch Plant','Part','Part Description','Standard Cost','Excess Qty_18 Wk']]

<ipython-input-11-0b2f929a06b7>:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rawExcess = rawExcess[conditions[0]]


In [ ]:
# Create new array for each individual plant's excess

# Group the 'excess' DataFrame by the 'Branch Plant' column and create a dictionary of DataFrames
df_dict = dict(tuple(excess.groupby('Branch Plant')))

excess1501 = df_dict[1501]
excess1502 = df_dict[1502]
excess1503 = df_dict[1503]
excess1504 = df_dict[1504]
excess1505 = df_dict[1505]
excess1506 = df_dict[1506]
excess1507 = df_dict[1507]
excess1508 = df_dict[1508]
excess1509 = df_dict[1509]
excess1510 = df_dict[1510]
excess1511 = df_dict[1511]
excess1512 = df_dict[1512]
excess1601 = df_dict[1601]
excess1602 = df_dict[1602]

In [ ]:
# Remove unneccesary columns from demand

demand = rawDemand[['2nd Item Number','Header Branch','Trans QTY','Request Date']]

In [ ]:
# group by 'Header Branch' and '2nd Item Number', and aggregate using sum and max functions

demand_sum = demand.groupby(['Header Branch', '2nd Item Number']).agg({'Trans QTY': 'sum', 'Request Date': 'min'}).reset_index()

In [ ]:
# separate all demand by branch plant

# group by 'Header Branch'

groups = demand_sum.groupby('Header Branch')

# create a new DataFrame for each group

branchdemands = []
for name, group in groups:
    branchdemands.append(group.copy())

In [ ]:
# separate demand into individual data frames

demand1501 = branchdemands[1]
demand1502 = branchdemands[2]
demand1503 = branchdemands[3]
demand1504 = branchdemands[4]
demand1505 = branchdemands[5]
demand1506 = branchdemands[6]
demand1507 = branchdemands[7]
demand1508 = branchdemands[8]
demand1509 = branchdemands[9]
demand1510 = branchdemands[10]
demand1511 = branchdemands[11]
demand1512 = branchdemands[12]
demand1601 = branchdemands[13]
demand1602 = branchdemands[14]

In [ ]:
# create ordered lists for sharing

order_1501 = [demand1509, demand1506, demand1511, demand1512, demand1601, demand1602, demand1503, demand1502, demand1508, demand1505, demand1504, demand1507, demand1510]
order_1502 = [demand1512, demand1601, demand1602, demand1503, demand1506, demand1509, demand1501, demand1505, demand1511, demand1508, demand1504, demand1507, demand1510]
order_1503 = [demand1508, demand1512, demand1506, demand1502, demand1509, demand1501, demand1505, demand1601, demand1602, demand1511, demand1504, demand1507, demand1510]
order_1504 = [demand1507, demand1510, demand1505, demand1502, demand1503, demand1512, demand1508, demand1601, demand1602, demand1506, demand1509, demand1501, demand1511]
order_1505 = [demand1503, demand1502, demand1508, demand1512, demand1504, demand1601, demand1507, demand1602, demand1506, demand1509, demand1501, demand1511, demand1510]
order_1506 = [demand1509, demand1501, demand1511, demand1512, demand1601, demand1602, demand1503, demand1502, demand1508, demand1505, demand1504, demand1507, demand1510]
order_1507 = [demand1504, demand1510, demand1505, demand1502, demand1503, demand1512, demand1601, demand1602, demand1508, demand1506, demand1509, demand1501, demand1511]
order_1508 = [demand1503, demand1512, demand1506, demand1509, demand1501, demand1505, demand1502, demand1601, demand1511, demand1602, demand1504, demand1507, demand1510]
order_1509 = [demand1506, demand1501, demand1511, demand1512, demand1601, demand1602, demand1503, demand1502, demand1508, demand1505, demand1504, demand1507, demand1510]
order_1510 = [demand1507, demand1504, demand1502, demand1505, demand1512, demand1602, demand1601, demand1503, demand1506, demand1509, demand1501, demand1511, demand1508]
order_1511 = [demand1501, demand1509, demand1506, demand1601, demand1602, demand1512, demand1503, demand1502, demand1508, demand1505, demand1504, demand1507, demand1510]
order_1512 = [demand1601, demand1602, demand1506, demand1509, demand1501, demand1502, demand1503, demand1511, demand1508, demand1505, demand1504, demand1507, demand1510]
order_1601 = [demand1602, demand1512, demand1509, demand1506, demand1501, demand1511, demand1502, demand1503, demand1508, demand1505, demand1504, demand1510, demand1507]
order_1602 = [demand1601, demand1512, demand1509, demand1506, demand1501, demand1511, demand1502, demand1503, demand1508, demand1505, demand1504, demand1510, demand1507]

order_1501 = order_1501[::-1]
order_1502 = order_1502[::-1]
order_1503 = order_1503[::-1]
order_1504 = order_1504[::-1]
order_1505 = order_1505[::-1]
order_1506 = order_1506[::-1]
order_1507 = order_1507[::-1]
order_1508 = order_1508[::-1]
order_1509 = order_1509[::-1]
order_1510 = order_1510[::-1]
order_1511 = order_1511[::-1]
order_1512 = order_1512[::-1]
order_1601 = order_1601[::-1]
order_1602 = order_1602[::-1]

In [ ]:
# create column order
cols = ['Branch Plant','Part','Part Description','Standard Cost','Excess Qty_18 Wk','Header Branch','Trans QTY','Request Date']

In [ ]:
# create loop that creates individual data frames for each part match from excess data frames to demand data frames

matches1501 = pd.DataFrame()


for df in order_1501:
    merged = pd.merge(excess1501, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1501 = pd.concat([merged, matches1501])
# rearrange dataframe for easier understanding

matches1501 = matches1501[cols]

# run again for the next plant

matches1502 = pd.DataFrame()


for df in order_1502:
    merged = pd.merge(excess1502, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1502 = pd.concat([merged, matches1502])
# rearrange dataframe for easier understanding

matches1502 = matches1502[cols]

# run again for the next plant

matches1503 = pd.DataFrame()


for df in order_1503:
    merged = pd.merge(excess1503, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1503 = pd.concat([merged, matches1503])
# rearrange dataframe for easier understanding

matches1503 = matches1503[cols]

# run again for the next plant

matches1504 = pd.DataFrame()


for df in order_1504:
    merged = pd.merge(excess1504, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1504 = pd.concat([merged, matches1504])
# rearrange dataframe for easier understanding

matches1504 = matches1504[cols]

# run again for the next plant

matches1505 = pd.DataFrame()


for df in order_1505:
    merged = pd.merge(excess1505, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1505 = pd.concat([merged, matches1505])
# rearrange dataframe for easier understanding

matches1505 = matches1505[cols]

# run again for the next plant

matches1506 = pd.DataFrame()


for df in order_1506:
    merged = pd.merge(excess1506, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1506 = pd.concat([merged, matches1506])
# rearrange dataframe for easier understanding

matches1506 = matches1506[cols]

# run again for the next plant

matches1507 = pd.DataFrame()


for df in order_1507:
    merged = pd.merge(excess1507, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1507 = pd.concat([merged, matches1507])
# rearrange dataframe for easier understanding

matches1507 = matches1507[cols]

# run again for the next plant

matches1508 = pd.DataFrame()


for df in order_1508:
    merged = pd.merge(excess1508, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1508 = pd.concat([merged, matches1508])
# rearrange dataframe for easier understanding

matches1508 = matches1508[cols]

# run again for the next plant

matches1509 = pd.DataFrame()


for df in order_1509:
    merged = pd.merge(excess1509, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1509 = pd.concat([merged, matches1509])
# rearrange dataframe for easier understanding

matches1509 = matches1509[cols]

# run again for the next plant

matches1510 = pd.DataFrame()


for df in order_1510:
    merged = pd.merge(excess1510, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1510 = pd.concat([merged, matches1510])
# rearrange dataframe for easier understanding

matches1510  = matches1510[cols]

# run again for the next plant

matches1511 = pd.DataFrame()


for df in order_1511:
    merged = pd.merge(excess1511, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1511 = pd.concat([merged, matches1511])
# rearrange dataframe for easier understanding

matches1511 = matches1511[cols]

# run again for the next plant

matches1512 = pd.DataFrame()


for df in order_1512:
    merged = pd.merge(excess1512, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1512 = pd.concat([merged, matches1512])
# rearrange dataframe for easier understanding


matches1512 = matches1512[cols]

# run again for the next plant

matches1601 = pd.DataFrame()


for df in order_1601:
    merged = pd.merge(excess1601, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1601 = pd.concat([merged, matches1601])
# rearrange dataframe for easier understanding

matches1601 = matches1601[cols]

# run again for the next plant

matches1602 = pd.DataFrame()


for df in order_1602:
    merged = pd.merge(excess1602, df, left_on='Part', right_on='2nd Item Number', how='inner')
    
    # Add the merged data to the results data frame
    matches1602 = pd.concat([merged, matches1602])
# rearrange dataframe for easier understanding

matches1602  = matches1602[cols]

matches = [matches1501,matches1502,matches1503,matches1504,matches1505,matches1506,matches1507,matches1508,matches1509,matches1510,matches1511,matches1512,matches1601,matches1602]

In [ ]:
# Loop over each unique part in the 'Part' column

#creating 1501s sharing report

combined_df = pd.DataFrame()

for part in matches1501['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1501[(matches1501['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1501 = combined_df[combined_df['taken']>0]

#creating 1502s sharing report

combined_df = pd.DataFrame()

for part in matches1502['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1502[(matches1502['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1502 = combined_df[combined_df['taken']>0]

#creating 1503s sharing report

combined_df = pd.DataFrame()

for part in matches1503['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1503[(matches1503['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1503 = combined_df[combined_df['taken']>0]

#creating 1504s sharing report

combined_df = pd.DataFrame()

for part in matches1504['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1504[(matches1504['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1504 = combined_df[combined_df['taken']>0]

#creating 1505s sharing report

combined_df = pd.DataFrame()

for part in matches1505['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1505[(matches1505['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1505 = combined_df[combined_df['taken']>0]

#creating 1506s sharing report

combined_df = pd.DataFrame()

for part in matches1506['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1506[(matches1506['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1506 = combined_df[combined_df['taken']>0]

#creating 1507s sharing report

combined_df = pd.DataFrame()

for part in matches1507['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1507[(matches1507['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1507 = combined_df[combined_df['taken']>0]

#creating 1508s sharing report

combined_df = pd.DataFrame()

for part in matches1508['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1508[(matches1508['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1508 = combined_df[combined_df['taken']>0]

#creating 1509s sharing report

combined_df = pd.DataFrame()

for part in matches1509['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1509[(matches1509['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1509 = combined_df[combined_df['taken']>0]

#creating 1510s sharing report

combined_df = pd.DataFrame()

for part in matches1510['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1510[(matches1510['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1510 = combined_df[combined_df['taken']>0]

#creating 1511s sharing report

combined_df = pd.DataFrame()

for part in matches1511['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1511[(matches1511['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1511 = combined_df[combined_df['taken']>0]

#creating 1512s sharing report

combined_df = pd.DataFrame()

for part in matches1512['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1512[(matches1512['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1512 = combined_df[combined_df['taken']>0]

#creating 1601s sharing report

combined_df = pd.DataFrame()

for part in matches1601['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1601[(matches1601['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1601 = combined_df[combined_df['taken']>0]

#creating 1602s sharing report

combined_df = pd.DataFrame()

for part in matches1602['Part'].unique():
    # Filter the data frame for the current part and matches1501
    part_df = matches1602[(matches1602['Part'] == part)]
    # Define the new columns
    taken_col = []
    leftover_col = []
    part_df = part_df.reset_index(drop=True)
    
    # Loop over each row in the filtered data frame
    for i, row in part_df.iterrows():
        if i == 0:
            # For the first row, calculate the 'taken' and 'leftover' values
            if row['Excess Qty_18 Wk'] > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = row['Excess Qty_18 Wk']
            taken_col.append(taken)
            leftover = row['Excess Qty_18 Wk'] - taken
            leftover_col.append(leftover)
        else:
            # For all following rows, calculate the 'taken' and 'leftover' values based on the previous row's 'leftover' value
            prev_leftover = leftover_col[-1]
            if prev_leftover > row['Trans QTY']:
                taken = row['Trans QTY']
            else:
                taken = prev_leftover
            taken_col.append(taken)
            leftover = prev_leftover - taken
            leftover_col.append(leftover)
    # Add the new columns to the filtered data frame
    part_df['taken'] = taken_col
    part_df['leftover'] = leftover_col
    combined_df = pd.concat([combined_df, part_df], ignore_index=True)
df1602 = combined_df[combined_df['taken']>0]

In [ ]:
# Put all the data frames into a list
dfs = [df1501, df1502, df1503, df1504, df1505, df1506, df1507, df1508, df1509, df1510, df1511, df1512, df1601, df1602]

# Concatenate the data frames along the rows (axis=0)
rawsharing = pd.concat(dfs, axis=0)

# Reset the index of the resulting data frame
rawsharing = rawsharing.reset_index(drop=True)

In [ ]:
# rename columns and clear requested value colun

rawsharing = rawsharing.rename( columns = {'Branch Plant':'From Plant','Header Branch':'To Plant','taken':'Requested Qty','leftover':'Proposed Value','Request Date':'MRP Demand Date'})
rawsharing['Proposed Value'] = ""

# multiply to find requested value

rawsharing['Proposed Value'] = rawsharing['Requested Qty'] * rawsharing['Standard Cost']

# drop any values where requested value is less than 100

rawsharing = rawsharing[rawsharing['Proposed Value'] >= 100]

# set column order

cols = ['Part','Part Description','From Plant','Requested Qty','Standard Cost','To Plant','MRP Demand Date','Proposed Value']

sharing = rawsharing[cols]

# set part to string to preserve 0s when exporting the CSV

sharing['Part'] = sharing['Part'].astype('|S')

# write the sharing to csv

sharing.to_csv('sharing.csv', index = False)
files.download('sharing.csv')

<ipython-input-22-a4fba8b6c4c0>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharing['Part'] = sharing['Part'].astype('|S')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
sharing.dtypes

Part                          |S15
Part Description            object
From Plant                   int64
Requested Qty              float64
Standard Cost              float64
To Plant                     int64
MRP Demand Date     datetime64[ns]
Proposed Value             float64
dtype: object